# Utilizando Baseline Dummy e ajustando escalas com StandardScaler

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [3]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"
dados = pd.read_csv(uri)
dados.head(3)

,Unnamed: 0,mileage_per_year,model_year,price,sold
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no


### Renomeando colunas

In [4]:
a_renomear = {
    'mileage_per_year' : 'milhas_por_ano',
    'model_year' : 'ano_do_modelo',
    'price' : 'preco',
    'sold' : 'vendido'
}
dados = dados.rename(columns=a_renomear)
dados.head(2)

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes


### Substituindo Yes ou No por 0 e 1

In [5]:
a_trocar = {
    'no' : 0,
    'yes' : 1
}
dados.vendido = dados.vendido.map(a_trocar)
dados.head(3)

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0


## Adicionando idade do modelo do carro

#### Assim, a diferença de idade fica mais analisavel
#### A diferença de 2000 e 2002 é so 2 para a escala 2.000

In [6]:
from datetime import datetime

ano_atual = datetime.today().year
dados['idade_do_modelo'] = ano_atual - dados.ano_do_modelo
dados.head(3)

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo
0,0,21801,2000,30941.02,1,24
1,1,7843,1998,40557.96,1,26
2,2,7109,2006,89627.50,0,18


### Convertendo milhas em km

In [7]:
dados['km_por_ano'] = dados.milhas_por_ano * 1.60934
dados.head(3)

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo,km_por_ano
0,0,21801,2000,30941.02,1,24,35085.22134
1,1,7843,1998,40557.96,1,26,12622.05362
2,2,7109,2006,89627.50,0,18,11440.79806


### Excluindo colunas desnecessarias

In [8]:
dados = dados.drop(columns = ["Unnamed: 0", "milhas_por_ano","ano_do_modelo"], axis=1)
dados.head(3)

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,24,35085.22134
1,40557.96,1,26,12622.05362
2,89627.50,0,18,11440.79806


## Separando as variaveis

In [9]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

## Separando treino e teste

In [10]:
SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))


Treinaremos com 7500 elementos e testaremos com 2500 elementos


### Instanciando, FIT e previsao (LinearSVC)

In [11]:
modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

C:\Users\leand\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### Acuracia

In [12]:
acuracia = accuracy_score(teste_y, previsoes) * 100
acuracia

46.88

# Dummy
#### Baseline

Faz previsões usando estratégias simples, como 'aleatoriedade', pra ver a eficacia de um modelo criado perante a comparacao

Tambem não preciso fazer a predição pra dps importar e fazer a acuracia, ja posso fazer isso direto com o score

In [13]:
from sklearn.dummy import DummyClassifier

In [14]:
# stratified = estrategia padrao - gera previsões aleatórias, mas mantém a mesma distribuição e proporcionalidade dos dados

dummy_stratified = DummyClassifier(strategy="stratified")
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 52.44%


### Estrategia mostfrequent

In [15]:
from sklearn.dummy import DummyClassifier

dummy_mostfrequent = DummyClassifier(strategy="most_frequent")
dummy_mostfrequent.fit(treino_x, treino_y)
acuracia = dummy_mostfrequent.score(teste_x, teste_y) * 100

print("A acurácia do dummy mostfrequent foi %.2f%%" % acuracia)

A acurácia do dummy mostfrequent foi 58.00%


# Utilizando o StandardScaler 
### 'Padronizando' as escalas

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

### Separando treino e teste

In [17]:
SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)

### StandardScaler

In [18]:
scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

## Instanciando e fazendo fit do modelo (SVC)

#### O LinearSVC, é menos sensível à escala dos dados e geralmente não requer padronização. 
####  O SVC se beneficia da padronização para garantir desempenho, especialmente quando há escalas diferentes.

In [19]:
modelo = SVC()
modelo.fit(treino_x, treino_y)

SVC()

### Previsao e acuracia

In [21]:
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
acuracia

77.48

### Notamos que houve um aumento significativo ao trabalhar a escala com o StandardScaler